In [28]:
from openai import OpenAI
import json
import os

GPT_MODEL_3_5 = "gpt-3.5-turbo-1106"
GPT_MODEL_4 = "gpt-4-0125-preview"

OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return json.loads(response.choices[0].message.content)

In [29]:
usecase_flow="""

"""

In [30]:
extracted_elements = """Elements: 

"""

In [31]:
SYSTEM_PROMPT_gen_test_scenarios ="""
I want you to work as a software engineering who has profound knowledge about software design and software testing.
You will be given a use case specification flow and a list of elements extracted from that use case specification flow.
Your task is to identify flows that at least one given element is used and mention the sentences where given element is used by user (not system display) in that flow.
Return the flows in json format.
The JSON format should follow the following structure:
{ "Flow name":   ["sentence mention given element in flow", "sentence mention given element in flow"]
  }
For example: {'Main flow': ['Step 1: Learner fills in the username field by a valid username that has been registered', 'Step 2: Learner fills in the password field by the correct password for the corresponding username'], 
              'Alternative flow 1: Login by email': ['At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered', 'Step 2: Learner fills in the password field by the correct password for the corresponding email'], 
              'Exception flow 1: Wrong password': ['At step 2 of the basic flow: Learner fills in the password field by a wrong password']}
"""

In [32]:
promptExtractCondtition = [
    { "role": "system", "content": SYSTEM_PROMPT_gen_test_scenarios},
    { "role": "user", "content": usecase_flow + extracted_elements }
  ]

gpt_response = ask(promptExtractCondtition, client, model)
print(gpt_response)

{'Main flow': ['Main flow: Step 1: Learner fills in the username field by a valid username that has been registered', 'Main flow: Step 2: Learner fills in the password field by the correct password for the corresponding username'], 'Alternative flow 1: Login by email': ['Alternative flow 1: Login by email: At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered', 'Alternative flow 1: Login by email: Step 2: Learner fills in the password field by the correct password for the corresponding email'], 'Exception flow 1: Wrong password': ['Exception flow 1: Wrong password: At step 2 of the basic flow: Learner fills in the password field by a wrong password'], 'Exception flow 2: Unregistered username': ['Exception flow 2: Unregistered username: At step 1 of the basic flow: Learner fills in the username field by a valid username that has not been registered'], 'Exception flow 3: Blank username fields': ['Exception flow 3: Blank username fields

In [35]:
filter_flow = []

for key, value in gpt_response.items():
    print(" ", key)
    print("Value:", value)

    if(value != [] ):
        filter_flow.append(key)



  Main flow
Value: ['Main flow: Step 1: Learner fills in the username field by a valid username that has been registered', 'Main flow: Step 2: Learner fills in the password field by the correct password for the corresponding username']
  Alternative flow 1: Login by email
Value: ['Alternative flow 1: Login by email: At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered', 'Alternative flow 1: Login by email: Step 2: Learner fills in the password field by the correct password for the corresponding email']
  Exception flow 1: Wrong password
Value: ['Exception flow 1: Wrong password: At step 2 of the basic flow: Learner fills in the password field by a wrong password']
  Exception flow 2: Unregistered username
Value: ['Exception flow 2: Unregistered username: At step 1 of the basic flow: Learner fills in the username field by a valid username that has not been registered']
  Exception flow 3: Blank username fields
Value: ['Exception flow

In [36]:
print(filter_flow)

['Main flow', 'Alternative flow 1: Login by email', 'Exception flow 1: Wrong password', 'Exception flow 2: Unregistered username', 'Exception flow 3: Blank username fields', 'Exception flow 3: Blank password fields']
